In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import sklearn.decomposition as sk
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression

import timeit
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

import json
import webbrowser

from sklearn.decomposition import NMF
from shapely.geometry import Point,LineString
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from collections import OrderedDict
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models import HoverTool

from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

Loading BokehJS ...

# Load data file

In [2]:
# path of csv files
# Municipal election data (2014)
vote2014 ='D:\\Jupyter_File\\Hackaton_Friday\\Dataset_haag\\voting_2014.csv'
# House of representatives election data (2017)
vote2017 = 'D:\\Jupyter_File\\Hackaton_Friday\\Dataset_haag\\House of Representatives 2017 - Districts.csv'

# convert csv to dataframe
df_ori_vote2014  = pd.read_csv(vote2014,index_col=None, header=0)
df_ori_vote2014['Wijken'] = df_ori_vote2014['Wijken'].apply(lambda x: x[3:])
df_ori_vote2014 = df_ori_vote2014.set_index('Wijken')
df_ori_vote2017  = pd.read_csv(vote2017,index_col=None, header=0)
df_ori_vote2017['Wijken'] = df_ori_vote2017['Wijken'].apply(lambda x: x[3:])
df_ori_vote2017 = df_ori_vote2017.set_index('Wijken')

# summing values of each districts
df_ori_vote2014['Total'] =  df_ori_vote2014.apply(lambda x: x.sum(), axis = 1)
df_ori_vote2017['Total'] =  df_ori_vote2017.apply(lambda x: x.sum(), axis = 1)

df_ori_vote2014 = df_ori_vote2014.reset_index()
df_ori_vote2017 = df_ori_vote2017.reset_index()

# fill the NaN values
df_vote2014 = df_ori_vote2014.fillna(0)
df_vote2017 = df_ori_vote2017.fillna(0)

# Check the quantity of the data
print('2014 Voting Data consists :',len(df_vote2014))
print('2017 Voting Data consists :',len(df_vote2017))
print('2014 Political Parties :',len(df_vote2014.columns)-1)
print('2017 Political Parties :',len(df_vote2017.columns)-1)

2014 Voting Data consists : 44
2017 Voting Data consists : 44
2014 Political Parties : 20
2017 Political Parties : 26


In [3]:
# # normalize the voting based on its value divided by total voting in that districts
# df_ori_vote2014_normalize = df_vote2014.loc[:,'PvdA':'TerLin'].div(df_vote2014['Total'], axis=0)
# df_ori_vote2017_normalize = df_vote2017.loc[:,' VVD':' VDP'].div(df_vote2017['Total'], axis=0)

# Checking the datatype in dataframe and fill NaN values

In [4]:
# # check how data type in dataframe
# print(df_ori_vote2014_normalize.dtypes.value_counts())
# print()
# print(df_ori_vote2017_normalize.dtypes.value_counts())

## Plot

In [5]:
# Set filepath (fix path relative to yours)
denhaag_shp_path = 'D:/Jupyter_File/Hackaton_Friday/Wijkgrenzen_Den_Haag/Wijkgrenzen_Den_Haag.shp'
denhaag_shp_data = gpd.read_file(denhaag_shp_path).sort_values(by='ID').set_index('ID').reset_index()
denhaag_shp_data.head()

,ID,WIJKCODE,WIJKNAAM,OPPERVLAKT,OMTREK,geometry
0,1,1,Oostduinen,3.085060e+06,7599.039509,"POLYGON ((4.288787753966882 52.1189587959923, ..."
1,2,2,Belgisch Park,9.729685e+05,4611.330485,"POLYGON ((4.281641354953662 52.11461065858767,..."
2,3,3,Westbroekpark en Duttendel,1.791382e+06,7086.181565,"POLYGON ((4.285673855620519 52.10421577656373,..."
3,4,4,Benoordenhout,2.744299e+06,9391.767964,"POLYGON ((4.306254131602312 52.09662956459162,..."
4,5,5,Archipelbuurt,8.712911e+05,4071.464096,"POLYGON ((4.293930625600745 52.0897924237977, ..."


In [6]:
denhaag_shp_data2 = denhaag_shp_data.copy()
combine = pd.concat([denhaag_shp_data2, df_vote2014], axis=1)
combine = combine.drop(['Wijken','Total'], axis=1)
combine.head()

,ID,WIJKCODE,WIJKNAAM,OPPERVLAKT,OMTREK,geometry,PvdA,PVV,VVD,D66,...,PvdD,ID,LP,CU/SGP,GdM/OP,HOOP,OPA,PvdE,FAM,TerLin
0,1,1,Oostduinen,3.085060e+06,7599.039509,"POLYGON ((4.288787753966882 52.1189587959923, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,Belgisch Park,9.729685e+05,4611.330485,"POLYGON ((4.281641354953662 52.11461065858767,...",208,412,572,517,...,66,6,3,70,158,22,1,4,9,0
2,3,3,Westbroekpark en Duttendel,1.791382e+06,7086.181565,"POLYGON ((4.285673855620519 52.10421577656373,...",55,118,282,180,...,27,1,0,28,29,18,1,3,1,1
3,4,4,Benoordenhout,2.744299e+06,9391.767964,"POLYGON ((4.306254131602312 52.09662956459162,...",345,338,2046,1520,...,106,7,11,92,108,34,1,1,4,6
4,5,5,Archipelbuurt,8.712911e+05,4071.464096,"POLYGON ((4.293930625600745 52.0897924237977, ...",195,172,426,516,...,62,5,8,37,49,11,1,1,5,2


In [7]:
def generate_random(number, polygon):
    list_of_points = []
    miny, minx, maxy, maxx = polygon.bounds
    counter = 0
    while counter < number:
        #pnt = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        pnt = Point(np.random.uniform(miny, maxy), np.random.uniform(minx, maxx))
        if polygon.contains(pnt):
            list_of_points.append(pnt)
            counter += 1
    return list_of_points

In [8]:
start_time = timeit.default_timer()
# Picked location between Sudbury and Somerville:
maploc = folium.Map(location=[52.1189587959923 ,4.288787753966882],zoom_start=12)

geojson_map = folium.GeoJson(combine.to_json()).add_to(maploc)

geojson_map.add_to(maploc)

color_list = ['#00308F','#AF002A','#3B7A57','#FF7E00','#C46210','#EFDECD','#9F2B68','#F19CBB','#804040','#391802'
             ,'#D3AF37','#34B334','#FF033E','#CFCFCF','#551B8C','#00FFFF','#8F9779','#9966CC','#FF2052','#007FFF'
             ,'#D3419D','#1DACD6','#F28E1C','#CAE00D','#8F5973','#660000','#DDE26A','#08E8DE','#66FF00','#BF94E4'
             ,'#964B00','#1B4D3E','#800020','#BD33A4','#536872','#FFF600','#00CC99','#ED9121','#ACE1AF','#DE3163']

for i in range(len(combine)):    
    for j in range(6,len(combine.columns)):
        if (combine.iloc[i,j] == 0):
            random_point = 0
        else:
            random_point = generate_random(combine.iloc[i,j],combine['geometry'][i])
            counter = combine.iloc[i,j]
            for point in range(0, len(random_point)):
                html_popup = folium.Html(
                                        """<h3>""" + """Citizen""" +""" - """+  str(counter) + """</h3><hr>""" +
                                        """ Vote for : """  + str(combine.columns[j]) +
                                        """<br>""" +
                                        """Profile : """ +
                                        """<br>""" +
                                        """- Tulip """ + (",".join(str(x) for x in 
                                                                   (random.sample([1, 2, 3, 4, 5, 6, 7, 8], 
                                                                                  random.randint(1, 4))))) +
                                        """<br>""" , script=True)
                popup_map = folium.Popup(html_popup, max_width=2650)
                folium.CircleMarker(tuple([random_point[point].y, random_point[point].x]),color=color_list[j],fill=True,
                                    fill_color=color_list[j],fill_opacity= 1,radius=3,popup=popup_map).add_to(maploc)
                counter = counter - 1

folium.LayerControl().add_to(maploc)

maploc.save('results.html') 
# Open new tab automatically for the html output that produced
webbrowser.open('results.html')

stop_time = timeit.default_timer()
print('total time for loop calculation: ', (stop_time-start_time)/60)

total time for loop calculation:  83.68526115631717
